In [9]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [ ]:
dataset = load_dataset("SetFit/SentEval-CR")

In [33]:
from datasets import Dataset

# Define your list of texts and labels
texts = ["I want to make an in-store purchase", "I will buy from store","make a purchase in person",
         "make a payment","redeem rewards","transaction declined","can i pay at store"]
labels = [1,1,1,0,0,0,0]  # Example labels (0 and 1)

# Create a dictionary with 'text' and 'label' keys
data = {"text": texts, "label": labels}

# Create a Dataset object from the dictionary
custom_dataset = Dataset.from_dict(data)

# Print the dataset
print(custom_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 7
})


In [34]:
# Select N examples per class (8 in this case)
#train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
#test_ds = dataset["test"]

train_ds = custom_dataset
test_ds = custom_dataset

In [35]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [36]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 280
  Num epochs = 1
  Total optimization steps = 18
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/18 [00:00<?, ?it/s]

***** Running evaluation *****


In [37]:
model.save_pretrained('ft1')

In [38]:
ft1 = SetFitModel.from_pretrained("ft1")

In [43]:
ft1.predict(["I will buy it from the Gap store myself","I will pay at the store","I want to redeem rewards",
             "help me in reedem store rewards","i will buy a new shirt from old navy","change atm pin"])

tensor([1, 0, 0, 0, 1, 0])